Pre-installed library imports

In [2]:

import importlib
import subprocess
import sys

Function to check if the libraries used in this code are already installed

In [3]:
def checkIfInstalled(library):
    try:
        importlib.import_module(library)
        print(f'{library} is installed')
    
    except ImportError:
        print(f'{library} is not installed, installing...')
        
        # Using sys.executable to install the library in the same python version, and -m to run the module as a script and ensure the correct version is used
        subprocess.run([sys.executable,'-m', 'pip', 'install', library])

Aditional library imports 

In [4]:
# Requests library, used to make HTTP requests
checkIfInstalled('requests')
import requests

requests is installed


TBA API setup

In [5]:
url = "https://www.thebluealliance.com/api/v3"
header = {"X-TBA-Auth-Key":"E5xeD6sWIPJe5QDAgtoXzoHkQpKxaoOJlyGXThQBH4SztWmnfEgSbjYLFbC40r1E"}

# Availability check
testResponse = requests.get(url + "/status", headers=header)

print(testResponse.status_code)

200


Fetch individual team data 
    -fetchRecursive: Recursively iterates through a list of dictionaries containing team data checking coincidences on a given team number, if there's a match, a dictionary containing team data is returned, else, an empty list is returned

    -fetchTeamData: Recursively extracts team lists from TBA API and uses fetchRecursive to find a team's data given a team number

In [10]:
# Helper function to recursively iterate through the teams using a team number as key
def fetchRecursive(teamsJson, teamNumber):
    if len(teamsJson) == 0:
        return []
    else:
        if (teamNumber == teamsJson[0]["team_number"]):
            return teamsJson[0] # Returns team dictionary
        else:
            return fetchRecursive(teamsJson[1:], teamNumber)

# Function to fetch team data by recursively iterating through the pages obtained from the API
def fetchTeamData(teamNumber, page = 0):
    teamsJson = requests.get(url + f"/teams/2023/{page}/simple", headers=header).json()
    
    if len(teamsJson) == 0:
        return []
    else:
        out = fetchRecursive(teamsJson, teamNumber)
        if type(out) == dict:
            return out 
        else:
            return fetchTeamData(teamNumber, page + 1)

# Test case
fetchTeamData(4403)    

{'city': 'Torreon',
 'country': 'Mexico',
 'key': 'frc4403',
 'name': 'PEÑOLES&PrepaTec Laguna',
 'nickname': 'PrepaTec - ROULT - Peñoles',
 'state_prov': 'Coahuila',
 'team_number': 4403}

FRC team class 

In [9]:
class Team:
    # Class Level attributes
    instances = []
    
    # Static Methods
    @staticmethod
    def fetchRecursive(teamsJson):
        if len(teamsJson) == 0:
            return []
        else:
            Team(teamsJson[0])
            return Team.fetchRecursive(teamsJson[1:])
               
    @staticmethod
    def fetchTeamData(page = 0):
        teamsJson = requests.get(url + f"/teams/2023/{page}/simple", headers=header).json()
        
        if len(teamsJson) == 0:
            return []
        else:
            Team.fetchRecursive(teamsJson)
            return Team.fetchTeamData(page + 1)
        
    # Class Methods
    @classmethod
    def instanceAllTeams(cls):
        Team.fetchTeamData()
    
    @classmethod
    def returnAllTeams(cls):
        return cls.instances
    
    # Constructor
    def __init__(self, teamDict):
        # Private atrributes
        self.__teamKey = teamDict["key"]
        
        # Public attributes
        self.teamNumber = teamDict["team_number"]
        self.name = teamDict["name"]
        self.city = teamDict["city"]
        self.state = teamDict["state_prov"]
        self.country = teamDict["country"]
        
        self.__class__.instances.append(self)
        
    @property
    def teamKey(self):
        return self.__teamKey
    
    def __repr__(self):
        return f"Team {self.teamNumber}"

Team.instanceAllTeams()


Team.returnAllTeams()



[Team 1,
 Team 4,
 Team 8,
 Team 11,
 Team 16,
 Team 20,
 Team 21,
 Team 25,
 Team 27,
 Team 28,
 Team 31,
 Team 33,
 Team 34,
 Team 41,
 Team 45,
 Team 48,
 Team 51,
 Team 56,
 Team 58,
 Team 59,
 Team 60,
 Team 61,
 Team 63,
 Team 66,
 Team 67,
 Team 68,
 Team 69,
 Team 70,
 Team 71,
 Team 74,
 Team 75,
 Team 78,
 Team 79,
 Team 85,
 Team 86,
 Team 87,
 Team 88,
 Team 93,
 Team 94,
 Team 95,
 Team 97,
 Team 100,
 Team 102,
 Team 103,
 Team 107,
 Team 108,
 Team 111,
 Team 112,
 Team 114,
 Team 115,
 Team 116,
 Team 117,
 Team 118,
 Team 120,
 Team 122,
 Team 123,
 Team 125,
 Team 126,
 Team 128,
 Team 131,
 Team 133,
 Team 135,
 Team 138,
 Team 141,
 Team 144,
 Team 148,
 Team 151,
 Team 155,
 Team 157,
 Team 159,
 Team 166,
 Team 167,
 Team 171,
 Team 172,
 Team 173,
 Team 175,
 Team 176,
 Team 177,
 Team 178,
 Team 179,
 Team 180,
 Team 181,
 Team 188,
 Team 190,
 Team 191,
 Team 192,
 Team 193,
 Team 195,
 Team 199,
 Team 201,
 Team 203,
 Team 204,
 Team 207,
 Team 216,
 Team 217,